# AI-Powered Contract Enrichment 🤖📜

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/lakelogic/LakeLogic/blob/main/examples/03_advanced_workflows/ai_contract_enrichment/ai_enrich_demo.ipynb) 
[![GitHub Repo](https://img.shields.io/badge/GitHub-Repo-blue?logo=github)](https://github.com/lakelogic/LakeLogic/blob/main/examples/03_advanced_workflows/ai_contract_enrichment/ai_enrich_demo.ipynb)


## Business Scenario

Your company has just acquired a startup. Their data team left behind **50+ CSV exports with zero documentation**. You need to:

1. **Understand** what each field means
2. **Identify** which fields contain PII (for GDPR/CCPA compliance)
3. **Write quality rules** before data goes into your lakehouse

**Without AI**: Weeks of manual work — interviewing former employees, reading SQL, guessing.

**With LakeLogic AI**: Minutes per dataset.

## Prerequisites

```bash
pip install "lakelogic[all]" openai
export OPENAI_API_KEY=sk-...

# Or use a local model (no API key needed):
# export LAKELOGIC_AI_PROVIDER=ollama
# export LAKELOGIC_AI_MODEL=llama3.1
```

## How It Works

The AI enricher sends **only schema + 5 sample rows** to the LLM. It never sends your full dataset. The LLM returns:

- Field descriptions
- PII flags with remediation suggestions
- SQL quality rules
- Domain and data layer classification

## Step 1: Load the Raw Contract

This contract has minimal rules — just `order_id IS NOT NULL` and `amount > 0`. A data engineer would normally need to spend hours adding descriptions, PII flags, and quality rules.

In [ ]:
from pathlib import Path
import yaml

# Resolve the example directory
def resolve_example_dir(name: str) -> Path:
    cwd = Path.cwd()
    for base in [cwd] + list(cwd.parents):
        candidates = [
            base / name,
            base / "examples" / "03_advanced_workflows" / name,
        ]
        for candidate in candidates:
            if candidate.exists():
                return candidate
    return cwd / name

base_path = resolve_example_dir("ai_contract_enrichment")
contract_path = base_path / "contract_ecommerce.yaml"

contract = yaml.safe_load(contract_path.read_text(encoding="utf-8"))

print("=" * 60)
print("BEFORE AI ENRICHMENT")
print("=" * 60)
print(f"Fields: {len(contract['model']['fields'])}")
print(f"Quality rules: {len(contract['quality']['row_rules'])}")
print(f"Fields with descriptions: 0")
print(f"Fields flagged as PII: 0")

## Step 2: Preview the Raw Data

Let's look at the CSV data that the AI will use to infer descriptions and rules.

In [ ]:
import pandas as pd

sample_df = pd.read_csv(base_path / "data" / "raw_orders.csv", nrows=10)

print(f"Sample data: {len(sample_df)} rows × {len(sample_df.columns)} columns")
print()
sample_df.head()

## Step 3: Enrich with AI

The enricher sends **schema + 5 sample rows** to the LLM. It returns:
- Human-readable field descriptions
- PII flags with remediation suggestions (hash, redact, mask)
- SQL quality rules the LLM infers from the data patterns
- Domain classification (e.g. "ecommerce") and data layer (e.g. "bronze")

In [ ]:
from lakelogic.ai.contract_enricher import enrich_contract

print("🤖 Calling AI for contract enrichment...")
print("   (sending schema + 5 sample rows — NOT your full dataset)")
print()

enriched = enrich_contract(
    contract,
    sample_df=sample_df,
    # provider="openai",       # default — change to "anthropic", "ollama", etc.
    # model="gpt-4o-mini",     # default — change to "gpt-4o", "claude-sonnet-4-20250514", etc.
)

print("✅ Enrichment complete!")

## Step 4: Review — Field Descriptions

The AI generated human-readable descriptions for every field. Compare this to the empty descriptions in the original contract.

In [ ]:
fields = enriched["model"]["fields"]
rules = enriched.get("quality", {}).get("row_rules", [])

n_descriptions = sum(1 for f in fields if f.get("description"))
n_pii = sum(1 for f in fields if f.get("pii"))

print("=" * 60)
print("AFTER AI ENRICHMENT")
print("=" * 60)
print(f"Fields: {len(fields)}")
print(f"Fields with descriptions: {n_descriptions}")
print(f"Fields flagged as PII: {n_pii}")
print(f"Quality rules: {len(rules)}")
print()

# Show field descriptions
print("─── Field Descriptions ─────────────────────────────────")
for f in fields:
    desc = f.get("description", "(none)")
    pii = "  🔒 PII" if f.get("pii") else ""
    classification = f" ({f['classification']})" if f.get("classification") else ""
    print(f"  {f['name']:<20} → {desc}{pii}{classification}")

## Step 5: Review — PII Detection & Remediation

The AI identified which fields contain Personally Identifiable Information and suggested remediation strategies (hash, redact, mask).

In [ ]:
pii_fields = [f for f in fields if f.get("pii")]

if pii_fields:
    print("─── PII Fields & Remediation ───────────────────────────")
    for f in pii_fields:
        remediation = f.get("pii_remediation", "not specified")
        print(f"  🔒 {f['name']:<20} type={f.get('classification', '?'):<12} action={remediation}")
else:
    print("No PII fields detected (try with real data)")

## Step 6: Review — AI-Suggested Quality Rules

The AI inferred SQL quality rules from the data patterns. These are rules a human would need hours to write — the AI suggests them in seconds.

In [ ]:
print("─── Quality Rules (SQL) ────────────────────────────────")
for rule in rules:
    if isinstance(rule, dict) and rule.get("sql"):
        desc = f"  # {rule['description']}" if rule.get("description") else ""
        print(f"  ✓ {rule['sql']}{desc}")

# Show metadata
metadata = enriched.get("metadata", {})
if metadata:
    print()
    print("─── Metadata ───────────────────────────────────────────")
    for k, v in metadata.items():
        print(f"  {k}: {v}")

## Step 7: Save the Enriched Contract

The enriched contract is a complete, production-ready YAML file with all descriptions, PII flags, and quality rules.

In [ ]:
output_path = base_path / "contract_ecommerce_enriched.yaml"
output_path.write_text(
    yaml.dump(enriched, sort_keys=False, allow_unicode=True, default_flow_style=False),
    encoding="utf-8",
)

print(f"✅ Enriched contract saved → {output_path.name}")
print()
print("Business impact:")
print("  • Field documentation: generated in seconds (vs hours manually)")
print("  • PII compliance: flagged automatically for GDPR/HIPAA review")
print("  • Quality rules: proactive protection before production incidents")
print()

# Preview the saved YAML
print("─── Preview of Enriched Contract ────────────────────────")
print(output_path.read_text(encoding="utf-8")[:1500])

## CLI Equivalent

Everything above can be done with a single CLI command:

```bash
lakelogic bootstrap \
    --landing data/ \
    --output-dir contracts/ \
    --registry contracts/reg.yaml \
    --ai \
    --ai-provider openai \
    --suggest-rules
```

The `--ai` flag triggers the same enrichment process shown in this notebook.